# KI in deinen Notebooks

https://github.com/ollama/ollama-python

In [ ]:
from ollama import chat
# default: http://127.0.0.1:11434 (Ollama läuft in der JupyterHub-VM)
# optional: http://10.0.2.2:11434 (Ollama läuft auf dem Host)

In [ ]:
%%bash
curl http://127.0.0.1:11434

So baust du KI in deine Notebooks ein:

In [ ]:
# Message senden
response = chat(
    model='gemma3:270m', 
    messages=[
        {'role': 'user', 'content': 'Why is the sky usually blue?'},
    ]
)

# Antwort komplett ausgeben
print(response.message.content)

Wenn du die Ausgabe Zeile für Zeile möchtest, geht das so:

In [ ]:
# Noch eine Nachricht an den Ollama-Server senden
stream = chat(
    model='gemma3:270m',
    messages=[
        {'role': 'user', 'content': 'Why is the sky sometimes red?'}
    ],
    stream=True,
)

# Diesmal die Antwort Zeile für Zeile ausgeben:
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

## Ollama in der JupyterHub-VM

Für kleine Modelle genügt die in der VM eingebaute Ollama-Instanz. Die verfügbaren Modelle lässt du dir so anzeigen:

In [ ]:
%%bash
ollama list

Informationen zu einem einzelenen MOdell, bspw. seine Grösse, erhälts du so:

In [ ]:
%%bash
ollama show gemma3:270m

Welche Modelle gerade ausgeführt werden, zeigt dir

In [ ]:
%%bash
ollama ps

Neue Modelle lädst du, indem du unter https://ollama.com/search dir eines aussuchst - bspw. `qwen3:0.6b` - und dann mit diesem Befehl herunterlädst:

In [ ]:
%%bash
ollama pull qwen3:0.6b &> output.txt
cat output.txt

Achtung, schau dir **VORHER** auf https://ollama.com/search genau an, wie gross ein Modell ist. Modelle über 1 GB werden in der VM kaum performant laufen. Wenn du ein grösseres Modell laufen lassen willst, nutzte dazu ein Ollama auf deinem Host-Rechner. (Siehe weiter unten)

Du löschst ein Modell mit diesem Befehl:

In [ ]:
%%bash
ollama rm NAME

Für NAME setze einen der Namen ein, den dir

In [ ]:
%%bash
ollama list

angezeigt hat.

## Ollama-Server vom Host nutzen

Du willst grössere Modell nutzen, die nicht in der VM laufen? Kein Problem!

**Wenn**

- [Ollama](https://ollama.com/download) auf dem Host läuft.
  - Vorteile: mehr RAM, CPU-Kerne und SSD-Speicher sowie ggf. Hardwarebeschleunigung durch deine GPU.
- **Und** du unter Settings "Expose Ollama to the Network" aktiviert hast.
- **Und** unter VirtualBox oder UTM eine passende Port-Weiterleitungs-Regel existiert.

**Dann**
- kannst du aus Jupyter unter der URL `http://10.0.2.2:11434` auf Ollama auf dem Host zugreifen.

Überprüfe ggf. die Weiterleitungsregeln in der VM unter: Ändern -> Netzwerk -> Port Weiterleitung

|Name|Protokoll|Host-IP|Host-Port|Gast-IP|Gast-Port|
|--|--|--|--|--|--|
|ollama-ext|TCP|10.0.2.2|11434|127.0.0.1|11434|

So testest du, ob der der Ollama-Server erreichbar ist:

In [ ]:
%%bash
curl http://10.0.2.2:11434

So nutzt du ein LLM, das nicht in der VM selbst läuft, sondern auf dem Host:

In [ ]:
from ollama import Client

In [ ]:
client = Client(
  host='http://10.0.2.2:11434'
)

Welche Modelle gibt es auf dem externen Ollama?

In [ ]:
def list_models(client):
    """Zeigt die verfügbares Modelle an"""
    for entry in client.list()['models']:
      print(entry["model"])

In [ ]:
list_models(client)

Jetzt kannst du wie gewohnt die Abfragen ausführen:

In [ ]:
response = client.chat(model='gemma3:270m', messages=[
  {
    'role': 'user',
    'content': 'What is the best Programming Language??',
  },
])

und die Antwort komplett ausgeben

In [ ]:
print(response.message.content)

Neue Modelle installierst entweder direkt auf dem Host oder von Jupyter aus:

In [ ]:
client.pull('qwen3:0.6b')

Teste auch diesen Befehl:

In [ ]:
client.ps()

und ggf. auch das Löschen eines (kleinen) Modells:

In [ ]:
list_models(client)

In [ ]:
client.delete('NAME')

In [ ]:
list_models(client)